<a href="https://colab.research.google.com/github/fukagai-takuya/gifu-ai/blob/main/gifu-ai-2025-12-21/FLUX1_schnell_GitClone_Diffusers_SaveGIF_old.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Google Drive のマウント (Google Drive 側で予め下記のコマンドを実行している環境を想定しています。)


git clone https://github.com/huggingface/diffusers.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

2. git clone した diffusers のソースコードを優先して参照するようにします

In [ ]:
import sys
sys.path.insert(0, "/content/drive/MyDrive/diffusers/src")

3. git clone した diffusers のソースコードが優先して参照されることを確認します

In [ ]:
import diffusers
print(diffusers.__file__)

4. diffusers を使用して FLUX.1 [schnell] を実行する際に必要なパッケージをインストールします

In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers accelerate safetensors
!pip install sentencepiece protobuf einops

5. FLUX.1 [schnell] を使用した Pipeline を用意します

In [ ]:
import torch
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell",
    torch_dtype=torch.float16
).to("cuda")


6. 下記のコードを実行し、画像を生成します。callback 関数を使用して denoising の各ステップ後の画像も出力するようにしています。

In [ ]:
from PIL import Image
import os

prompt = 'The phrase "Gifu AI Study Group" formed from small colorful geometric shapes.'

height = 1024
width = 1024

out_dir = "steps"
os.makedirs(out_dir, exist_ok=True)

# 生成途中の画像を追加する GIF ファイル用の空のリスト
gif_frames = []

def callback(pipe, step_index, timestep, callback_kwargs):
    latents = callback_kwargs["latents"]

    with torch.no_grad():

        latents = pipe._unpack_latents(
            latents,
            height,
            width,
            pipe.vae_scale_factor,
        )

        image = pipe.vae.decode(
            latents / pipe.vae.config.scaling_factor
        ).sample

        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.cpu().permute(0, 2, 3, 1).numpy()

        img = Image.fromarray((image[0] * 255).astype("uint8"))
        img.save(f"steps/step_{step_index:03d}.png")
        gif_frames.append(img)

    return callback_kwargs


pipe(
    prompt,
    height=height,
    width=width,
    guidance_scale=0.0,
    num_inference_steps=4,
    max_sequence_length=256,
    callback_on_step_end=callback
)


# =====  GIF を作成 =====
gif_path = "Gifu-AI-Study-Group.gif"

gif_frames[0].save(
    gif_path,
    save_all=True,
    append_images=gif_frames[1:],
    duration=500,  # ms / frame
    loop=0
)

print(f"GIF saved to {gif_path}")

